In [1]:
#normalizing
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
df_scaled = min_max_scaler.fit_transform(df)
df_normalized = pd.DataFrame(df_scaled, columns = cols)

#transformation
from feature_engine import transformation
X = df.iloc[:, 0:9].copy()
y = df['state']
tf = transformation.YeoJohnsonTransformer(variables = ['actual_soc', 'consumption'])
tf.fit(X)
X = tf.transform(X)

#smote
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(X, y)
X = pd.concat([X_smote, y_smote], axis=1)

#onehotencoding
from sklearn.preprocessing import OneHotEncoder
X1 = X.copy()
X1 = X1.values
enc = OneHotEncoder(categories='auto')
X1 = enc.fit_transform(X1).toarray()


#train test split
#10 fold cv with grid search
del X1['state']
X_train, X_test, y_train, y_test = train_test_split(X1, y_smote, test_size=0.25, random_state=1)

#model training and tuning
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True)
model = DecisionTreeClassifier()
k_values = [1,2,3,5,10]
searchspace = {'max_depth': k_values}
search = GridSearchCV(model, searchspace=searchspace, scoring='accuracy', cv=kf, refit=True, verbose=0, n_jobs=-1)
result = search.fit(X_train, y_train)

In [2]:
workflow = setup(df, target = 'state', normalize = True, transformation = True, transformation_method = 'yeo-johnson', 
                 fix_imbalance = True, session_id = 123)
dt = create_model('dt')
k_values = [1,2,3,5,10]
searchspace = {'max_depth': k_values}
tuned_dt = tune_model(dt, custom_grid = searchspace)

In [3]:
eda(display_format = 'svg')

In [4]:
dashboard(dt)

In [5]:
evaluate_model(dt)